<img src="./img/Logo.jpg" align="right" width="150" /> 

## Medidas Electrónicas 1

# Trabajo Práctico de Laboratorio Nº 2: Error de Método

### **Curso:** 
* R4052 2023

### **Docentes:**
* Marinsek, Emiliano
* Perdomo, Juan Manuel

### **Nº de Grupo:** 
* 2
###   **Integrantes:**
* Borello, Federico
* Dimaio, Agustin
* Manoukian, Francisco Tomas
* Mella, Camila
* Ronchetti Juan Manuel

# Objetivo
En esta practica, se pide obtener la tension de salida de un divisor resistivo en dos casos: en primer lugar teniendo en cuenta un error de metodo despreciable, y en segundo lugar generando artificialmente un error de metodo del 20%.

El error de metodo es aquel atribuible al metodo con el cual se realiza la medicion, en este caso mediante la utilizacion de un voltimetro. La resistencia interna del instrumento afecta la medicion, y se puede calcular el error de metodo de la siguiente manera:

<img src="./img/eMR.png"  width=350 /> 

Llamando $V_v$ a la tension entre A y B sin el instrumento y $V_I$ a la tension entre A y B con el instrumento, resulta:
* $V_v = V_{TH}$
* $V_I = V_{RV}$

$V_{RV} = V_{TH} \cdot \frac{R_V}{R_{TH} + R_V}$

$e_{MR} = \frac{V_I - V_v}{V_v} = \frac{V_{RV} - V_{TH}}{V_{TH}}$

Se deduce que: 

$|e_{MR}| = \frac{|R_{TH}|}{|R_{TH} + R_{RV}|}$

# Especificaciones

A continuacion se detallan las especificaciones de los instrumentos y componentes utilizados:

#### Especificaciones del multimetro:
* Error medicion de tension: $0.5\% + 2d$
* Error medicion resistencia: $0.8\% + 2d$
* Display: $3  \frac{1}{2}$

#### Especificaciones de la fuente: 
* Modelo: HY3005D
* Error display de tension: $1\% + 2d$
* Datasheet: [Enlace](https://www.diverseelectronics.com/upload/documents/Mastech-HY3003.pdf)

#### Especificaciones de los resistores: 
* Tolerancia: $5\%$ 

## Inicializacion de bibliotecas

In [2]:
# Biblioteca pandas para visualizar los datos
import pandas as pd
import numpy as np
import sympy as sp
import math as m
pd.set_option('display.max_rows', None)

# Desarrollo

## Ensayo numero 1
El circuito a evaluar es el siguiente:

<img src="./img/Circuito1.png"  width=250 /> 

Que aplicando Teorema de Thevenin resulta:

$V_{TH} = V_1 \cdot \frac{R_B}{R_B + R_A}$

$R_{TH} = R_A//R_B$

### Mediciones de Ra y Rb

Se tomaron mediciones de Ra y Rb con el multimetro en modo medicion de resistencia, y se obtuvieron los siguientes resultados:

In [21]:
datosR = pd.read_excel("./Mediciones/TP2.xlsx")
display(datosR)

Ra = datosR['Ra [kOhm]'].tolist()
Rb = datosR['Rb [kOhm]'].tolist()

R_a = np.average(Ra) * 1000
R_b = np.average(Rb) * 1000
R_th = (R_a * R_b)/(R_a + R_b)

print("\nValores mas comunes:\n")
print("Ra = ", R_a)
print("Rb = ", R_b)
print("Rth = ",R_th)

,Ra [kOhm],Rb [kOhm]
0,9.81,9.91
1,9.82,9.92
2,9.81,9.92
3,9.81,9.91
4,9.82,9.91



Valores mas comunes:

Ra =  9814.000000000002
Rb =  9913.999999999998
Rth =  4931.873276561233


$R_{TH} = R_A//R_B = 4.932k\Omega$

<img src="./img/Circuito2.png"  width=250 /> 

A la hora de medir, la resistencia del voltimetro posee un valor de $R_V = 10M\Omega$, por ende se tiene:

<img src="./img/Circuito3.png"  width=250 /> 

El error de metodo resulta entonces:

$e_{MR} = |\frac{-R_{TH}}{R_{TH} + R_V}| = |\frac{-4.932k\Omega}{4.932k\Omega + 10M\Omega}|$

In [23]:
e_mr = R_th/(R_th + 10e6)

print("e_mr = ", e_mr)

e_mr =  0.0004929442138166275


$e_{MR} = |\frac{-R_{TH}}{R_{TH} + R_V}| = 493.2\cdot10^{-6}$

Para determinar que el error de metodo sea despreciable, se toma como criterio que $e_{MR} < \frac{U_R}{10}$, por ende es necesario obtener la $U_R$ de la medicion de tension en $R_b$.

Siendo la incertidumbre tipo B:

$u_b = (\frac{e_{r\%}}{100} + \frac{n}{c})\cdot \frac{1}{\sqrt(3)} $

A continuacion se muestran las mediciones tomadas para la $V_{R_b}$ y su incertidumbre:

In [28]:
VRb = np.array([2.55, 2.54, 2.55, 2.54, 2.55])
V_Rb = np.average(VRb)
print("V_Rb = ", V_Rb)

# Tipo A
ua_Vrb = ( (1/m.sqrt( len(VRb) )) * np.std(VRb) )
# Tipo B
ub_Vrb = ( (1/m.sqrt(3)) * V_Rb * (0.5/100 + 2/255) )
# Combinada
u_Vrb = m.sqrt(ua_Vrb**2 + ub_Vrb**2)

U_VRb = u_Vrb * 2  
print("U_VRb = ", U_VRb)

print("U_R_VRb / 10 = ",U_VRb/(10*V_Rb))

print("e_mr = ", e_mr)

V_Rb =  2.5460000000000003
U_VRb =  0.03801052901812003
U_R_VRb / 10 =  0.0014929508648122555
e_mr =  0.0004929442138166275


Se verifica que $e_{MR} < \frac{U_R}{10}$ por ende el error de metodo es despreciable.

## Ensayo numero 2

En este ensayo se buscó generar un error de metodo del 20%. Para eso se parte de la medicion de tension en $R_b$ cuando el error de metodo es despreciable, la cual fue calculada en el ensayo anterior.

Con un error de metodo del 20%, se calcula que el valor que se deberia obtener es aproximadamente $2.04V$. 

In [26]:
print("V_Rb con error de metodo despreciable = ", V_Rb)
print("V_Rb con error de metodo 20% = ", V_Rb * 0.8)

V_Rb con error de metodo despreciable =  2.5460000000000003
V_Rb con error de metodo 20% =  2.0368000000000004


### Procedimiento
Se colocó un potenciometro en paralelo con $R_B$, y se ajustó su valor midiendo la tension de $R_B$ hasta que el voltimetro indicara $2.04V$. 

Una vez obtenido ese valor, se retiró el voltimetro y se midio el valor de resistencia del potenciometro. 

#### Valor indicado por el potenciometro:

$R_p = 19.75k\Omega$

<img src="./img/Circuito4.png"  width=250 /> 

Finalmente, se verifica numericamente el error de metodo obtenido:

$e_{MR} = |\frac{-R_{TH}}{R_{TH} + (R_V // R_p)}| = |\frac{-4.932k\Omega}{4.932k\Omega + (10M\Omega // 19.75k\Omega)}| $

In [27]:
R_p = 19.75 * 1000

e_mr2 = R_th / (R_th + ( (R_p * 10e6)/(R_p + 10e6) ))
print("e_mr2 = ", e_mr2)

e_mr2 =  0.20013328335128014


Se confirma que el ensayo fue exitoso, y se obtuvo el error de metodo buscado de manera artificial.

# Conclusiones
Se concluye que la experiencia fue exitosa: se estudiaron los efectos del error de metodo en la medicion, se observaron en la practica y se pudieron calcular dos errores de metodos distintos para el mismo circuito, alterando artificialmente la medicion en el segundo caso.